## part1: 数据处理

In [58]:
# # Credit: sentdex
# # link: https://pythonprogramming.net/3d-convolutional-neural-network-machine-learning-tutorial/
# import os
# import pandas as pd

# data_dir = 'data/demo/nodule/'
# patients = os.listdir(data_dir)
# labels_df = pd.read_csv('data/demo/train_data.csv',index_col=0)

# labels_df.head()

In [59]:
# len(patients)

In [1]:
# import matplotlib.pyplot as plt
# import numpy as np
# import math

In [60]:
# 验证数据集的大小
# for patient in patients[:8]:
#     patient_name = patient[0:-4]
#     label = labels_df.get_value(patient_name, 'lable')
#     path = data_dir + patient
#     # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
#     scan = np.load(path)
#     print(scan['voxel'],label)
#     plt.figure()
#     plt.subplot(1,2,1)
#     plt.imshow(scan['voxel'][37])
#     plt.subplot(1,2,2)
#     plt.imshow(scan['seg'][37])

In [61]:
# 将数据集存储在muchdata.npy中：
# much_data = []
# for num, patient in enumerate(patients):
#     patient_name = patient[0:-4]
#     label = labels_df.get_value(patient_name, 'lable')
#     path = data_dir + patient
#     img_data = np.load(path)
#     voxel = img_data['voxel'].astype(np.int32)
# #     plt.imshow(voxel[50])
# #     print(label)
#     much_data.append([voxel,label])
    
# np.save('muchdata', much_data)

 ## part2: 神经网络模块
 some codes work below, follow the tutorial: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count() )

True
2


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import math

### 怎么把数据我现在的数据变成需要的数据？

- 先要理清思路，现在的数据长什么样子?
source https://blog.csdn.net/DaVinciL/article/details/78793067 介绍了CIFAR数据集
- 现在的网络每层的输入输出是什么样子？

### train part:

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input_channel:1; output_channel(#filter):10; kernel_size: 10*10
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=5, kernel_size=17)
        # maxpool: kernel: 2; stride: 2
        self.pool = nn.MaxPool3d(2, 2)
        # input_channel:10; output_channel(#filter):20; kernel_size: 10*10
        self.conv2 = nn.Conv3d(in_channels=5, out_channels=10, kernel_size=17)
        self.fc1 = nn.Linear(10 * 13 * 13 * 13, 16000)
        self.fc2 = nn.Linear(16000, 8000)
        self.fc3 = nn.Linear(8000, 2000)
        self.fc4 = nn.Linear(2000, 200)
        self.fc5 = nn.Linear(200, 80)
        self.fc6 = nn.Linear(80, 10)
        self.fc7 = nn.Linear(10, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
#         x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 10 * 13 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.softmax(self.fc7(x)) # 变成概率形式
        return x


net = Net()
net.cuda()

Net(
  (conv1): Conv3d(1, 5, kernel_size=(17, 17, 17), stride=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(5, 10, kernel_size=(17, 17, 17), stride=(1, 1, 1))
  (fc1): Linear(in_features=21970, out_features=16000, bias=True)
  (fc2): Linear(in_features=16000, out_features=8000, bias=True)
  (fc3): Linear(in_features=8000, out_features=2000, bias=True)
  (fc4): Linear(in_features=2000, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=80, bias=True)
  (fc6): Linear(in_features=80, out_features=10, bias=True)
  (fc7): Linear(in_features=10, out_features=2, bias=True)
)

In [6]:
import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
# use np.load to import data, devide dataset into 2 parts: train_data & validation_data:
# credit: cheez & Matthew Kerian
# link: https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56243777
'''use older to successfully load the data:'''
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

data_ineed = np.load('muchdata.npy')

# restore np.load for future normal usage
np.load = np_load_old

# If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
train_data = data_ineed[:-60]
validation_data = data_ineed[-60:]

In [7]:
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_data, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs: tensor(100*(100*100)), label
        inputs_numpy, labels = data
        inputs = torch.from_numpy(inputs_numpy)
        inputs = inputs.unsqueeze(0)
        inputs = inputs.unsqueeze(0)
        inputs = inputs.float()
        inputs = inputs.cuda()
        labels = torch.from_numpy(np.asarray(labels)).float()
        labels = labels.long() #credit: ptrblck; link: https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542/4
        labels = labels.unsqueeze(0)
        labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs = torch.log(outputs) # 为了放进损失函数 NLLLoss() 中：

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

/home/vivi/anaconda3/envs/3dunet/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,    20] loss: 0.694
[1,    40] loss: 0.692
[1,    60] loss: 0.691
[1,    80] loss: 0.691
[1,   100] loss: 0.704
[1,   120] loss: 0.700
[1,   140] loss: 0.694
[1,   160] loss: 0.698
[1,   180] loss: 0.700
[1,   200] loss: 0.695
[1,   220] loss: 0.686
[1,   240] loss: 0.703
[1,   260] loss: 0.703
[1,   280] loss: 0.693
[1,   300] loss: 0.691
[1,   320] loss: 0.694
[1,   340] loss: 0.679
[1,   360] loss: 0.690
[1,   380] loss: 0.674
[1,   400] loss: 0.688
[2,    20] loss: 0.706
[2,    40] loss: 0.714
[2,    60] loss: 0.714
[2,    80] loss: 0.697
[2,   100] loss: 0.692
[2,   120] loss: 0.689
[2,   140] loss: 0.670
[2,   160] loss: 0.701
[2,   180] loss: 0.708
[2,   200] loss: 0.697
[2,   220] loss: 0.678
[2,   240] loss: 0.711
[2,   260] loss: 0.710
[2,   280] loss: 0.689
[2,   300] loss: 0.690
[2,   320] loss: 0.695
[2,   340] loss: 0.672
[2,   360] loss: 0.690
[2,   380] loss: 0.670
[2,   400] loss: 0.689
[3,    20] loss: 0.709
[3,    40] loss: 0.719
[3,    60] loss: 0.699
[3,    80] 

In [8]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

### test part

In [10]:
PATH = './cifar_net.pth'
net = Net()
net.load_state_dict(torch.load(PATH))
net.cuda()

Net(
  (conv1): Conv3d(1, 5, kernel_size=(17, 17, 17), stride=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(5, 10, kernel_size=(17, 17, 17), stride=(1, 1, 1))
  (fc1): Linear(in_features=21970, out_features=16000, bias=True)
  (fc2): Linear(in_features=16000, out_features=8000, bias=True)
  (fc3): Linear(in_features=8000, out_features=2000, bias=True)
  (fc4): Linear(in_features=2000, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=80, bias=True)
  (fc6): Linear(in_features=80, out_features=10, bias=True)
  (fc7): Linear(in_features=10, out_features=2, bias=True)
)

In [45]:
predict_value = []
true_value = []
for i, data in enumerate(validation_data, 0):
    inputs_numpy, labels = data
    inputs = torch.from_numpy(inputs_numpy)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.float()
    inputs = inputs.cuda()
    labels = torch.from_numpy(np.asarray(labels)).float()
    labels = labels.long() #credit: ptrblck; link: https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542/4
    labels = labels.unsqueeze(0)
    labels = labels.cuda()

    outputs = net(inputs)

    outputs = outputs.cpu()
    outputs = outputs.squeeze(0)
    outputs = outputs.detach().numpy()
    predict_value.append(outputs[1])
    
    labels = labels.cpu()
    labels = labels.detach().numpy()
    labels[0]
    true_value.append(labels[0])
    
    # ROC-AUC reference: https://www.cnblogs.com/dlml/p/4403482.html

/home/vivi/anaconda3/envs/3dunet/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [48]:
from sklearn.metrics import roc_auc_score
roc_auc_score(true_value, predict_value)